## Cervical Cancer Behavior Risk

The dataset contains 19 attributes regarding ca cervix behavior risk with class label is ca_cervix with 1 and 0 as values which means the respondent with and without ca cervix, respectively.

In [209]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import copy, math
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler


In [210]:
# Load data from .data file 
data = pd.read_csv('./cervical+cancer+behavior+risk/sobar-72.csv')

In [211]:
#  Calculating the percentage of correctly classified examples
# 
def PercentCorrect(Inputs, targets, weights):
    N = len(targets)
    nCorrect = 0
    for n in range(N):
        OneInput = Inputs[n,:]
        if (targets[n]*np.dot(OneInput, weights) > 0):
            nCorrect += 1 
    return 100*nCorrect/N 

In [212]:
# Prepare data set as we want and devide to train and test sets

targets = []
targets = data.iloc[:, 19].values
print(targets.shape)

for i in range(len(targets)):
    if(targets[i] == 0):
        targets[i] = -1

X = data.iloc[:, 0:18+1]

NumDataPerClass = 36

rIndex = np.random.permutation(X.shape[0])  # Shuffle row indices
Xr = X.iloc[rIndex]  # Shuffle rows of X
yr = targets[rIndex]  # Corresponding labels after shuffling

X_train = Xr[0:NumDataPerClass]
y_train = yr[0:NumDataPerClass]
X_test = Xr[NumDataPerClass:2*NumDataPerClass]
y_test = yr[NumDataPerClass:2*NumDataPerClass]
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

Ntrain = NumDataPerClass
Ntest = NumDataPerClass

X_train = X_train.values
X_test = X_test.values


# Create a StandardScaler object
scaler = StandardScaler()

# Fit the scaler to your training data (learn mean and standard deviation)
X_train_scaled = scaler.fit_transform(X_train)
# print(X_train_scaled)

# Apply the scaling to your test data (using the parameters learned from training data)
X_test_scaled = scaler.transform(X_test)
# print(X_test_scaled)


(72,)
(36, 19) (36,) (36, 19) (36,)


In [213]:
# Random initialization of weights
w = np.random.randn(19)

# What is the performance with the initial random weights?
print('Initial Percentage Correct: %6.2f' %(PercentCorrect(X_train_scaled, y_train, w)))


Initial Percentage Correct:  44.44


In [214]:
def sigmoid(z):

    # g = 1/(1+np.exp(-z))
    
    # Clip the output of the sigmoid function to avoid numerical instability
    g = np.clip(1 / (1 + np.exp(-z)), 1e-15, 1 - 1e-15)
   
    return g

In [215]:
def compute_gradient_logistic(X, y, w, b, lambda_): 
        
    m,n = X.shape
    dj_dw = np.zeros((n,))                           #(n,)
    dj_db = 0.
    
    for i in range(m):
        z = np.dot(X[i],w) + b
        f_wb_i = sigmoid(z)
        err_i = f_wb_i - y[i]
        for j in range(n):
            dj_dw[j] = dj_dw[j] + err_i * X[i,j] 
        dj_db = dj_db + err_i
    dj_dw = dj_dw/m
    dj_db = dj_db/m
    
    for j in range(n):
        dj_dw[j] = dj_dw[j] + (lambda_/m)*w[j]
    
    return dj_db, dj_dw

In [216]:
def compute_cost_logistic(X, y, w, b, lambda_ = 1.5):
    m,n  = X.shape
    cost = 0.0
    for i in range(m):
        z_i = np.dot(X[i],w) + b
        f_wb_i = sigmoid(z_i)
        cost +=  -y[i]*np.log(f_wb_i) - (1-y[i])*np.log(1-f_wb_i)         
    cost = cost / m
    
    reg_cost = 0
    for j in range(n):
        reg_cost += (w[j]**2)                                          #scalar
    reg_cost = (lambda_/(2*m)) * reg_cost                              #scalar
    
    total_cost = cost + reg_cost                                       #scalar
    return total_cost

In [217]:
def gradient_descent(X, y, w_in, b_in, alpha, num_iters, lambda_):
    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    w = copy.deepcopy(w_in)  #avoid modifying global w within function
    b = b_in 
    
    for i in range(num_iters):
        
        dj_db, dj_dw = compute_gradient_logistic(X, y, w, b, lambda_)
        
        w = w - alpha * dj_dw
        b = b - alpha * dj_db 
        
        # Save cost J at each iteration
        J_history.append( compute_cost_logistic(X, y, w, b) )
        
        if i% math.ceil(num_iters / 10) == 0:
            print(f"Iteration {i:4d}: Cost {J_history[-1]}   ")
    
    return w, b, J_history  


In [218]:
poly_features = PolynomialFeatures(degree=3)

# Fit and transform the training data
X_train_poly = poly_features.fit_transform(X_train_scaled)

# Transform the test data using the same transformation learned from the training data
X_test_poly = poly_features.transform(X_test_scaled)


w_tmp  = np.zeros_like(X_train_poly[0])
b_tmp  = 0.
alph = 0.001
iters = 2000
lambda_tmp = 1.5

w_out, b_out, _ = gradient_descent(X_train_poly, y_train, w_tmp, b_tmp, alph, iters, lambda_tmp) 
print(f"\nupdated parameters: w:{w_out}, b:{b_out}")

print('Percentage Correct After Training: %6.2f %6.2f'%(PercentCorrect(X_train_poly, y_train, w_out), PercentCorrect(X_test_poly, y_test, w_out)))

Iteration    0: Cost 0.5237330803090755   


Iteration  200: Cost -12.093404188629405   
Iteration  400: Cost -15.467528152504288   
Iteration  600: Cost -16.358051220006395   
Iteration  800: Cost -16.908036002578356   
Iteration 1000: Cost -17.238129914950125   
Iteration 1200: Cost -17.37968334919296   
Iteration 1400: Cost -17.154917995665777   
Iteration 1600: Cost -16.659894969888377   


C:\Users\User\AppData\Local\Temp\ipykernel_17132\3919815716.py:6: RuntimeWarning: overflow encountered in exp
  g = np.clip(1 / (1 + np.exp(-z)), 1e-15, 1 - 1e-15)


Iteration 1800: Cost -15.984178366660815   

updated parameters: w:[-1.19838566 -0.26880962  0.2784805  ... -0.08599835  0.02548317
  0.45743047], b:-1.2492307024123819
Percentage Correct After Training:  97.22  83.33
